# Домашнее задание 2
## Named Entity Recognition and Event Extraction

deadline: 06 ноября 2022, 23:59

В этом домашнем задании вы будете работать с двумя корпусами - LitBank и MultiNERD_EN.
Первый корпус (LitBank) собран из популярных художественных произведений на английском языке и содержит разметку по именованным сущностям и событиям, корпус состоит из 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещения), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глаголом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 


Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,

Второй корпус (MultiNERD_EN) состоит из 164 тысяч предложений по различным тематикам. От предыдущего корпуса он отличается большим числом (15) типов именованных сущностей. Также одной из сущностей является сущность-событие (EVE). Каждая сущность может состоять как из одного, так и нескольких слов.

Корпус доступен в репозитории проекта:  https://github.com/Babelscape/multinerd

В статьях и репозиториях вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и использовать повторно. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо в ipython-тетрадке. При этом прикладывать ipynb-файл обязательно. 
43. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросовестное цитирование приводит к обнулению оценки. 


## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиения. Авторы статей предлагают следующую структуру разбиения для корпуса LitBank: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.
Для корпуса MultiNERD_EN произведите стратифицированное разбиение по предложениям в соотношении 80%:10%:10%. Стратификацию производить в отношении пропорции по именованным сущностям (понятно, что с учётом всех вводных идеального разбиения не получится, но старайтесь сохранить распределения по типам сущностей для каждой и подвыборок).


## Часть 2. [3 балла] Извлечение именованных сущностей
Для каждой корпуса необходимо осуществить раздельно следующую процедуру:
1. [0.75 балла] Обучите стандартную модель для извлечения именованных сущностей, CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей. 
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация).
2. [0.75 балла] Замените часть модели на символах и словах (CNN + эмбеддинги словах) на ELMo и / или BERT. Должна получиться модель ELMo / BERT + BiLSTM + CRF. 
3. [1.5 балла] Замените модель последовательности (BiLSTM) на другой слой, например, на Transformer. Должна получиться модель CNN  + Transformer + CRF.
В результате у вас будет 2 набора моделей - проверьте качество извлечения на тестовых данных.
- Примените модели первого корпуса ко всем (!!!) предложениям второго корпуса и наоборот.
- Составьте отчёт по качеству работы моделей в терминах извлечения по типам сущностей, которые встречаются в обоих корпусах. Метрику выберите самостоятельно.
- Проанализируйте ошибки моделей в терминах предсказания сущностям первого корпуса специфических типов для второго корпуса. И наоборот - когда специфическим типам второго корпуса модель, обученная на LitBank, прсваивает одну из своих категорий.

[бонус] Дообучите BERT для извлечения именованных сущностей.

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]

[бонус] Модифицируйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018]: вместо эмбеддингов слов используйте ELMo и/или BERT. 

## Часть 3. [2 балла] Извлечение событий 

1. [0.75 балла] Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий (на основе LitBank). 

2. [0.75 балла] Замените часть модели на словах на ELMo и/или BERT. Должна получиться модель ELMo / BERT + BiLSTM.

3. [0.5 балла] Проверьте "извлекающую" силу модели на данных MultiNERD_EN для событий типа EVE. 

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров. 

[бонус] Дообучите BERT для извлечения триггеров событий. 

## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий 
1. [0.75 балла] Обучите модель для совместного извлечения именованных сущностей и триггеров событий только на основе LitBank (!). У модели должен быть общий энкодер (например, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именованных сущностей, другой отвечает за извлечение триггеров событий.

2. [0.75 балла] Создайте единую обучающую подвыборку, единую валидационную и единую тестовую на основе разбиений, которые вы произвели в первой части и обучите модель. (Единую - то есть на основе обоих корпусов сразу)

3. [0.5 балла] Сравните предсказательную силу модели из п.1 и модели из п.2 как на совместных подвыборках, так и на раздельных. Проанализируйте полученный результат. Приводит ли обогащение дополнительными данными к улучшению "извлекающей" способности модели?

[бонус] Добавьте в модель механизм внимания, таким способом, который покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания (attention). 

## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

<span style="color:pink">Авторы работы: Токкожин Арсен, Соколов Ян, Екимов Егор, Гвасалия Лукас</span>
# Решение:

## Часть 1. [2 балла] Эксплоративный анализ 
1. [0.4 балла] Найдите топ 10 (по частоте) именованных сущностей каждого из типов для каждого корпуса. Произведите сравнение частотности по совпадающим сущностям при сопоставлении корпусов друг с другом.
2. [0.4 балла] Найдите топ 10 (по частоте) частотных триггеров событий (на основе LitBank). Произведите анализ из пункта 1 для именованной сущности типа EVE (на основе MultiNERD_EN). 
3. [0.6 балла] Кластеризуйте все уникальные триггеры событий (LitBank), используя эмбеддинги слов и любой алгоритм кластеризации (например, аггломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий? 
4. [0.6 балла] Отдельно произведите анализ из п. 3 для сущностей типа EVE (на основе MultiNERD_EN). Сравните результаты из п. 3 и п. 4.

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу LitBank и сравните кластеры триггеров и выделенные темы: есть ли схожие паттерны в тематической модели и в структуре кластеров?

In [22]:
import os
from brat_parser import get_entities_relations_attributes_groups
from collections import Counter

entities_folder_path = "litbank_entities/brat"
events_folder_path = "litbank_events/brat"
multinerd_path = "multinerd_en.tsv"

#### Топ 10 именованных сущностей для корпуса Litbank

In [23]:
entities = ["PER", "FAC", "LOC", "ORG", "GPE", "VEH"]

In [24]:
def count_entities(path_to_folder, entities_names):
    result = {}
    for entity in entities_names:
        result[entity] = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                result[entities[item].type][entities[item].text] += 1 

    return result

In [65]:
entities_ans = count_entities(entities_folder_path, entities)
for entity in entities_ans:
    print("-------------------------------------")
    print(entity)
    temp = entities_ans[entity].most_common()[:10]
    for item in temp:
        print(item)

-------------------------------------
PER
('Mr.', 148)
('Miss', 133)
('Mrs.', 132)
('sir', 50)
('Sir', 45)
('my mother', 40)
('men', 40)
('Cameron', 38)
('his wife', 37)
('Mr', 37)
-------------------------------------
FAC
('home', 65)
('the house', 52)
('there', 39)
('here', 39)
('the room', 34)
('the garden', 23)
('the street', 14)
('the hall', 13)
('the road', 13)
('the place', 12)
-------------------------------------
LOC
('the world', 72)
('the sea', 27)
('the river', 22)
('the country', 20)
('there', 18)
('the earth', 16)
('sea', 16)
('the valley', 13)
('this world', 12)
('the woods', 9)
-------------------------------------
ORG
('the army', 7)
('the Committee of Public Safety', 4)
('the Colonial Office', 4)
('the Church', 4)
('Harvard', 3)
('college', 3)
('the hospital', 2)
('the C.C.H.', 2)
("the Bank of Leichardt 's Land", 2)
('a regiment of regulars', 2)
-------------------------------------
GPE
('London', 40)
('England', 32)
('there', 21)
('the town', 21)
('New York', 16)
('

#### Топ 10 именованных сущностей для корпуса MultiNERD_EN

In [26]:
import pandas as pd
import csv
col_names = ["Index", "Token", "Label", "BabelNet ID", "Wikidata ID", "Wikipedia ID", "Wikipedia Title", "Definition", "Image URL", "Smth"]
multinerd_data = pd.read_csv(multinerd_path, sep="\t", names=col_names, index_col=False , quoting=csv.QUOTE_NONE)

In [27]:
multinerd_data.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL,Smth
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...,NaN
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(multinerd_data["Smth"].size)
print(multinerd_data["Smth"].isna().count())

3568155
3568155


видим что последняя колонка полнстью состоит из пропусков, так что ее можно удалить

In [29]:
multinerd_data=multinerd_data.drop(["Smth"], axis=1)

In [30]:
multinerd_data.head()

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,B-PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...
3,3,Cameron,I-PER,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
multinerd_data.loc[multinerd_data['Label'] != 'O', 'Label'] = multinerd_data.loc[multinerd_data['Label'] != 'O', 'Label'].str[2:]

In [33]:
multinerd_data.head(10)

,Index,Token,Label,BabelNet ID,Wikidata ID,Wikipedia ID,Wikipedia Title,Definition,Image URL
0,0,Created,O,NaN,NaN,NaN,NaN,NaN,NaN
1,1,by,O,NaN,NaN,NaN,NaN,NaN,NaN
2,2,James,PER,bn:00786804n,Q42574,15622.0,James_Cameron,James Francis Cameron is a Canadian film direc...,https://upload.wikimedia.org/wikipedia/commons...
3,3,Cameron,PER,NaN,NaN,NaN,NaN,NaN,NaN
4,4,and,O,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Charles,PER,bn:02320763n,Q3666404,2956611.0,Charles_H._Eglee,Charles Hamilton Eglee is an American film and...,https://upload.wikimedia.org/wikipedia/commons...
6,6,H.,PER,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Eglee,PER,NaN,NaN,NaN,NaN,NaN,NaN
8,8,",",O,NaN,NaN,NaN,NaN,NaN,NaN
9,9,it,O,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
all_labels = list(multinerd_data["Label"])
cnt = Counter()
for label in all_labels:
    cnt[label] += 1
cnt

Counter({'O': 3111887,
         'PER': 155669,
         'LOC': 112519,
         'MEDIA': 19673,
         'TIME': 5584,
         'EVE': 8525,
         'ORG': 78179,
         'PLANT': 12674,
         'ANIM': 22284,
         'FOOD': 15006,
         'DIS': 18713,
         'CEL': 4341,
         'SUPER': 774,
         'VEHI': 1114,
         'INST': 822,
         'PHY': 190,
         'BIO': 201})

In [68]:
multinerd_entities_names = cnt.keys()

In [49]:
df = multinerd_data.groupby(['Label', 'Token'])[['Index']].count().reset_index()

In [61]:
df = df.rename(columns={
    "Index":"Amount",
})

In [69]:
multinerd_entities = {}
for entity in multinerd_entities_names:
    print("-------------------------------------")
    print(entity)
    temp = df[df["Label"] == entity].sort_values(["Amount"], ascending=False)
    print(temp[["Token","Amount"]].head(10).to_string(index=False))
    multinerd_entities[entity] = list(temp["Token"].head(10))

-------------------------------------
O
Token  Amount
    ,  194114
  the  169553
    .  164196
   of   94883
  and   90076
   in   83052
    "   72596
    a   65926
   to   58243
   is   40965
-------------------------------------
PER
  Token  Amount
   John    2097
 George    1114
  David    1026
William     911
  James     879
Michael     824
 Robert     794
   Paul     745
Charles     691
Richard     660
-------------------------------------
LOC
    Token  Amount
   United    2357
      New    2030
   States    1604
    South    1568
Australia    1103
  America    1035
       of     999
    North     998
   Africa     943
    River     919
-------------------------------------
MEDIA
   Token  Amount
Tomatoes    1466
  Rotten    1466
     The     954
      of     498
     the     452
      's     168
       :     131
     You     119
     and     108
    Show     107
-------------------------------------
TIME
      Token  Amount
        Day     222
        Age     214
     Middle   

In [70]:
multinerd_entities

{'O': [',', 'the', '.', 'of', 'and', 'in', '"', 'a', 'to', 'is'],
 'PER': ['John',
  'George',
  'David',
  'William',
  'James',
  'Michael',
  'Robert',
  'Paul',
  'Charles',
  'Richard'],
 'LOC': ['United',
  'New',
  'States',
  'South',
  'Australia',
  'America',
  'of',
  'North',
  'Africa',
  'River'],
 'MEDIA': ['Tomatoes',
  'Rotten',
  'The',
  'of',
  'the',
  "'s",
  ':',
  'You',
  'and',
  'Show'],
 'TIME': ['Day',
  'Age',
  'Middle',
  'Ages',
  'Great',
  'Depression',
  'Renaissance',
  'of',
  'Christmas',
  'Bronze'],
 'EVE': ['War',
  'World',
  'II',
  'Civil',
  'Cup',
  'American',
  'Revolution',
  'Festival',
  'League',
  'Film'],
 'ORG': ['University',
  'of',
  'United',
  'Party',
  'City',
  'New',
  'FC',
  'National',
  'Town',
  'Union'],
 'PLANT': ['plant',
  'rice',
  'oak',
  'wheat',
  'pepper',
  'shrub',
  'deciduous',
  '"',
  'tree',
  'pine'],
 'ANIM': ['moth',
  'sea',
  'snail',
  'fish',
  'deer',
  'trout',
  'Erebidae',
  'Noctuidae',


In [72]:
litbank_entities = {}
for entity in entities_ans:
    temp = entities_ans[entity].most_common()[:10]
    litbank_entities[entity] = list()
    for item in temp:
        litbank_entities[entity].append(item[0])

In [73]:
litbank_entities

{'PER': ['Mr.',
  'Miss',
  'Mrs.',
  'sir',
  'Sir',
  'my mother',
  'men',
  'Cameron',
  'his wife',
  'Mr'],
 'FAC': ['home',
  'the house',
  'there',
  'here',
  'the room',
  'the garden',
  'the street',
  'the hall',
  'the road',
  'the place'],
 'LOC': ['the world',
  'the sea',
  'the river',
  'the country',
  'there',
  'the earth',
  'sea',
  'the valley',
  'this world',
  'the woods'],
 'ORG': ['the army',
  'the Committee of Public Safety',
  'the Colonial Office',
  'the Church',
  'Harvard',
  'college',
  'the hospital',
  'the C.C.H.',
  "the Bank of Leichardt 's Land",
  'a regiment of regulars'],
 'GPE': ['London',
  'England',
  'there',
  'the town',
  'New York',
  'town',
  'France',
  'Europe',
  'the country',
  'Rome'],
 'VEH': ['the ship',
  'the car',
  'the train',
  'boats',
  'the boat',
  'a carriage',
  'the waggon',
  'the carriage',
  'the coach',
  'the Fuwalda']}

In [76]:
for entity in list(set(litbank_entities) & set(multinerd_entities)):
    print("----------------------------")
    print(entity)
    print("Litbank:")
    print(litbank_entities[entity])
    print("Multinerd:")
    print(multinerd_entities[entity])

----------------------------
PER
Litbank:
['Mr.', 'Miss', 'Mrs.', 'sir', 'Sir', 'my mother', 'men', 'Cameron', 'his wife', 'Mr']
Multinerd:
['John', 'George', 'David', 'William', 'James', 'Michael', 'Robert', 'Paul', 'Charles', 'Richard']
----------------------------
LOC
Litbank:
['the world', 'the sea', 'the river', 'the country', 'there', 'the earth', 'sea', 'the valley', 'this world', 'the woods']
Multinerd:
['United', 'New', 'States', 'South', 'Australia', 'America', 'of', 'North', 'Africa', 'River']
----------------------------
ORG
Litbank:
['the army', 'the Committee of Public Safety', 'the Colonial Office', 'the Church', 'Harvard', 'college', 'the hospital', 'the C.C.H.', "the Bank of Leichardt 's Land", 'a regiment of regulars']
Multinerd:
['University', 'of', 'United', 'Party', 'City', 'New', 'FC', 'National', 'Town', 'Union']


Сравнение: одинаковые сущности в разных датасетах трактуются по-разному. Видим что в датасете multinerd большую популярность среди сущностей персон(PER) и локаций(LOC) имеют имена собственные, нежели в датасете litbank.

#### Топ 10 частотных триггеров событий для корпуса Litbank

In [77]:
def count_events(path_to_folder):
    cnt = Counter()

    for filename in os.listdir(path_to_folder):
        if filename.endswith(".ann"):
            entities, relations, attributes, groups = get_entities_relations_attributes_groups(os.path.join(path_to_folder, filename))
            for item in entities:
                cnt[entities[item].text] += 1 

    return cnt

In [80]:
litbank_events = count_events(events_folder_path).most_common()[:10]
litbank_events

[('said', 464),
 ('came', 95),
 ('looked', 92),
 ('went', 92),
 ('asked', 69),
 ('heard', 63),
 ('cried', 59),
 ('saw', 59),
 ('took', 56),
 ('turned', 55)]

In [81]:
multinerd_entities["EVE"]

['War',
 'World',
 'II',
 'Civil',
 'Cup',
 'American',
 'Revolution',
 'Festival',
 'League',
 'Film']

Сравневние: видим что тут тоже по-разному трактуется понятие event. В litbank это больше определяется по совершенному общему действию, а в multinerd это больше похоже на какое-то конкретное событие.